**Importing libraries**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

**Getting the dataset**

In [ ]:
df = pd.read_csv("insurance_data.csv")
df.head(7)

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0


In [ ]:
#splitting the data into test and train
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

In [ ]:
X_train.head(3)

,age,affordibility
0,22,1
13,29,0
6,55,0


In [ ]:
len(X_train)

22

In [ ]:
max(X_train['age'])#to scale the data I am taking max here

62

**Scaling the data**

In [ ]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/62

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/62

In [ ]:
X_train_scaled.head(5)

,age,affordibility
0,0.354839,1
13,0.467742,0
6,0.887097,0
17,0.935484,1
24,0.806452,1


In [ ]:
X_test_scaled.head(5)

,age,affordibility
2,0.758065,1
10,0.290323,1
21,0.419355,0
11,0.451613,1
14,0.790323,1


**Building the neural network**

In [ ]:
model = keras.Sequential(
    keras.layers.Dense(1,input_shape = (2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
    #here kernel initializer is used to set the initial weights and bias initializer for initial bias
)
model.compile(optimizer='adam',
              loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train_scaled,Y_train,epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 4ms/step - loss: 0.5273 - accuracy: 0.8182
Epoch 2502/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.5272 - accuracy: 0.8182
Epoch 2503/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5272 - accuracy: 0.8182
Epoch 2504/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5271 - accuracy: 0.8182
Epoch 2505/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5271 - accuracy: 0.8182
Epoch 2506/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5270 - accuracy: 0.8182
Epoch 2507/5000
1/1 [==============================] - 0s 15ms/step - loss: 0.5270 - accuracy: 0.8182
Epoch 2508/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5270 - accuracy: 0.8182
Epoch 2509/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5269 - accuracy: 0.8182
Epoch 2510/5000
1/1 [==============================] -

In [ ]:
#evaluating the model
model.evaluate(X_test_scaled,Y_test)

1/1 [==============================] - 0s 438ms/step - loss: 0.3178 - accuracy: 1.0000


[0.31784626841545105, 1.0]

In [ ]:
model.predict(X_test_scaled)

array([[0.7199068 ],
       [0.30199537],
       [0.16735315],
       [0.44438222],
       [0.7440038 ],
       [0.85865766]], dtype=float32)

In [ ]:
#values of weights and bias after training
coef , intercept = model.get_weights()
coef,intercept

(array([[3.8093946],
        [1.2582239]], dtype=float32), array([-3.2019913], dtype=float32))

**Now building a neural network without using tensorflow**


In [ ]:
#sigmoid function
import math
def sigmoid(x):
  return 1/(1+math.exp(-x))
sigmoid(7)

0.9990889488055994

In [ ]:
#prediction function
def prediction_function(age,affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability + intercept
  return sigmoid(weighted_sum)

In [ ]:
prediction_function(0.758065,1)

0.719907219811121

In [ ]:
#loss function
def log_loss(y_true,y_predicted):
    #since log(0) is infinite and their might be a chance that we might get log(0) we are using epsilon
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    #now changing this list to numpy array
    y_predicted_new = np.array(y_predicted_new)
    return np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))


In [ ]:
#sigmoid function but for a sequence of values
def sigmoid_numpy(X):
  return 1/ (1+np.exp(-X))
sigmoid_numpy(np.array([16,0,1]))

array([0.99999989, 0.5       , 0.73105858])

In [ ]:
class mynn:
  def __init__(self):
    self.w1=1
    self.w2=1
    self.bias=0
  def fit(self,X,y,epochs):
    self.w1 , self.w2 , self.bias = gradient_descent(X['age'] , X['affordibility'] , y ,epochs)
  def predict(self,X_test):
    weighted_sum = self.w1* X_test['age'] + self.w2*X_test['affordibility'] + self.bias
    return sigmoid_numpy(weighted_sum)
  def gradient_descent(age,affordability,y_true,epochs):
    w1=w2=1
    bias=0
    lrate = 0.01
    n= len(affordability)
    for i in range(epochs):
      weighted_sum = w1*age + w2*affordability + bias
      y_predicted = sigmoid_numpy(weighted_sum)
      
      loss = log_loss(y_true,y_predicted)
      
      w1dt = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
      w2dt = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
      biasdt = np.mean(y_predicted-y_true)

      w1 = w1 - lrate*w1dt
      w2 = w2 - lrate*w2dt
      bias = bias - lrate*biasdt
      if i%50==0:
        print(f'Epoch:{i} , weight1 : {w1} , weight2 : {w2} , bias : {bias} , loss : {loss}')
    return w1,w2,bias

In [ ]:
custommodel = mynn()
custommodel.fit(X_train_scaled,Y_train,epochs=1231)

Epoch:0 , weight1 : 0.9988928445296062 , weight2 : 0.9987378420554621 , bias : -0.002684034594711748 , loss : -0.7479069342274475
Epoch:1 , weight1 : 0.9977899112191699 , weight2 : 0.9974796224079692 , bias : -0.005361391784401525 , loss : -0.7469060242831496
Epoch:2 , weight1 : 0.996691198650463 , weight2 : 0.9962253422469313 , bias : -0.008032075245100861 , loss : -0.7459106157044268
Epoch:3 , weight1 : 0.99559670532675 , weight2 : 0.9949750026745071 , bias : -0.010696088768978406 , loss : -0.7449206909081628
Epoch:4 , weight1 : 0.9945064296730234 , weight2 : 0.9937286047057117 , bias : -0.013353436263900348 , loss : -0.7439362322262224
Epoch:5 , weight1 : 0.9934203700362433 , weight2 : 0.992486149268531 , bias : -0.016004121752982877 , loss : -0.742957221906916
Epoch:6 , weight1 : 0.9923385246855846 , weight2 : 0.991247637204045 , bias : -0.018648149374136794 , loss : -0.7419836421164614
Epoch:7 , weight1 : 0.9912608918126884 , weight2 : 0.990013069266558 , bias : -0.021285523379604

In [ ]:
custommodel.predict(X_test_scaled)

2     0.638883
10    0.518683
21    0.348123
11    0.561122
14    0.646733
9     0.692077
dtype: float64